# End to End Machine Learning Pipeline for Income Prediction with Seldon Deploy

We use [demographic features from the 1996 US census](https://archive.ics.uci.edu/ml/datasets/census+income) to build an end to end machine learning pipeline. The pipeline is also annotated so it can be run as a [Kubeflow Pipeline](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) using the [Kale](https://github.com/kubeflow-kale/kale) pipeline generator.

The notebook/pipeline stages are:

 1. Setup 
   * Imports
   * pipeline-parameters
   * minio client test
 1. Train a simple sklearn model and push to minio
 1. Prepare an Anchors explainer for model and push to minio
 1. Test Explainer
 1. Train an isolation forest outlier detector for model and push to minio
 1. Deploy a Seldon model and test using the Seldon Deploy Enterprise API
 1. Deploy an outlier detector with the Seldon Deploy Enterprise API
 1. Test the outlier detector



In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult
from minio import Minio
from minio.error import ResponseError
from joblib import dump, load
import dill
import time
import json
from subprocess import run, Popen, PIPE
from alibi_detect.utils.data import create_outlier_batch
import swagger_client
from swagger_client.rest import ApiException
import yaml
import json
import urllib3
urllib3.disable_warnings()

In [2]:
MINIO_HOST="minio-service.kubeflow:9000"
MINIO_ACCESS_KEY="minio"
MINIO_SECRET_KEY="minio123"
MINIO_MODEL_BUCKET="seldon"
INCOME_MODEL_PATH="sklearn/income/model"
EXPLAINER_MODEL_PATH="sklearn/income/explainer"
OUTLIER_MODEL_PATH="sklearn/income/outlier"
DEPLOY_NAMESPACE="admin"
DEPLOY_SERVER = "https://x.x.x.x/seldon-deploy/"
DEPLOY_USER = "admin@kubeflow.org"
DEPLOY_PASSWORD = "12341234"

In [3]:
def get_minio():
    return Minio(MINIO_HOST,
                    access_key=MINIO_ACCESS_KEY,
                    secret_key=MINIO_SECRET_KEY,
                    secure=False)

In [4]:
def get_swagger_configuration():
    configuration = swagger_client.Configuration()
    configuration.host = 'http://seldon-deploy.seldon-system/seldon-deploy/api/v1alpha1'
    return configuration

In [5]:
import requests

from urllib.parse import urlparse

KF_SESSION_COOKIE_NAME = "authservice_session"


class SessionAuthenticator:
    """
    Returns the cookie token.
    """

    def __init__(self, server: str):
        self._server = server

        url = urlparse(server)
        self._host = f"{url.scheme}://{url.netloc}"

    def authenticate(self, user: str, password: str) -> str:
        auth_path = self._get_auth_path()
        success_path = self._submit_auth(auth_path, user, password)
        session_cookie = self._get_session_cookie(success_path)
        return session_cookie

    def _get_auth_path(self) -> str:
        # Send unauthenticated request
        res = requests.get(self._server, allow_redirects=False, verify=False)

        # Follow the 302 redirect
        oidc_path = res.headers["Location"]
        oidc_endpoint = f"{self._host}{oidc_path}"
        res = requests.get(oidc_endpoint, allow_redirects=False, verify=False)

        return res.headers["Location"]

    def _submit_auth(self, auth_path: str, user: str, password: str) -> str:
        auth_endpoint = f"{self._host}{auth_path}"
        auth_payload = {"login": user, "password": password}
        res = requests.post(auth_endpoint, auth_payload, allow_redirects=False, verify=False)
        
        login_path = res.headers["Location"]
        login_endpoint = f"{self._host}{login_path}"
        res = requests.get(login_endpoint, allow_redirects=False, verify=False)

        return res.headers["Location"]

    def _get_session_cookie(self, success_path: str) -> str:
        success_endpoint = f"{self._host}{success_path}"
        res = requests.get(success_endpoint, allow_redirects=False, verify=False)
        print(res.cookies)
        return res.cookies[KF_SESSION_COOKIE_NAME]

def authenticate():
    authenticator = SessionAuthenticator(DEPLOY_SERVER)

    cookie = authenticator.authenticate(DEPLOY_USER, DEPLOY_PASSWORD)
    return cookie


In [6]:
minioClient = get_minio()
buckets = minioClient.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mlpipeline 2020-11-24 08:48:44.210000+00:00
seldon 2020-11-25 14:29:28.707000+00:00


In [7]:
if not minioClient.bucket_exists(MINIO_MODEL_BUCKET):
    minioClient.make_bucket(MINIO_MODEL_BUCKET)

## Train Model

In [8]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [9]:
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

Note that for your own datasets you can use our utility function [gen_category_map](../api/alibi.utils.data.rst) to create the category map:

In [10]:
from alibi.utils.data import gen_category_map

Define shuffled training and test set

In [11]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:,:-1]
target = data_perm[:,-1]

In [12]:
idx = 30000
X_train,Y_train = data[:idx,:], target[:idx]
X_test, Y_test = data[idx+1:,:], target[idx+1:]

### Create feature transformation pipeline
Create feature pre-processor. Needs to have 'fit' and 'transform' methods. Different types of pre-processing can be applied to all or part of the features. In the example below we will standardize ordinal features and apply one-hot-encoding to categorical features.

Ordinal features:

In [13]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())])

Categorical features:

In [14]:
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Combine and fit:

In [15]:
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features),
                                               ('cat', categorical_transformer, categorical_features)])

### Train Random Forest model

Fit on pre-processed (imputing, OHE, standardizing) data.

In [16]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)

In [17]:
model=Pipeline(steps=[("preprocess",preprocessor),("model",clf)])
model.fit(X_train,Y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 8, 9, 10]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [1, 2, 3, 4, 5, 6, 7, 11])]

Define predict function

In [18]:
def predict_fn(x):
    return model.predict(x)

In [19]:
#predict_fn = lambda x: clf.predict(preprocessor.transform(x))
print('Train accuracy: ', accuracy_score(Y_train, predict_fn(X_train)))
print('Test accuracy: ', accuracy_score(Y_test, predict_fn(X_test)))

Train accuracy:  0.9655333333333334
Test accuracy:  0.855859375


In [20]:
dump(model, 'model.joblib') 

['model.joblib']

In [21]:
print(get_minio().fput_object(MINIO_MODEL_BUCKET, f"{INCOME_MODEL_PATH}/model.joblib", 'model.joblib'))

('1f63c9adf7f8ae03a3e363e910b5a83a-7', None)


## Train Explainer

In [22]:
model.predict(X_train)
explainer = AnchorTabular(predict_fn, feature_names, categorical_names=category_map)

Discretize the ordinal features into quartiles

In [23]:
explainer.fit(X_train, disc_perc=[25, 50, 75])

AnchorTabular(meta={'name': 'AnchorTabular', 'type': ['blackbox'], 'explanations': ['local'], 'params': {'seed': None, 'disc_perc': [25, 50, 75]}})

In [24]:
with open("explainer.dill", "wb") as dill_file:
    dill.dump(explainer, dill_file)    
    dill_file.close()
print(get_minio().fput_object(MINIO_MODEL_BUCKET, f"{EXPLAINER_MODEL_PATH}/explainer.dill", 'explainer.dill'))

('2800a0f978a6b88af3d86cdb91fa4eac-2', None)


## Get Explanation

Below, we get an anchor for the prediction of the first observation in the test set. An anchor is a sufficient condition - that is, when the anchor holds, the prediction should be the same as the prediction for this instance.

In [28]:
model.predict(X_train)
idx = 0
class_names = adult.target_names
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

Prediction:  <=50K


We set the precision threshold to 0.95. This means that predictions on observations where the anchor holds will be the same as the prediction on the explained instance at least 95% of the time.

In [29]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: Marital Status = Separated AND Relationship = Unmarried AND Capital Gain <= 0.00
Precision: 0.96
Coverage: 0.18


## Train Outlier Detector

In [30]:
from alibi_detect.od import IForest

od = IForest(
    threshold=0.,
    n_estimators=200,
)


In [31]:
od.fit(X_train)

In [32]:
np.random.seed(0)
perc_outlier = 5
threshold_batch = create_outlier_batch(X_train, Y_train, n_samples=1000, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
#X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

5.0% outliers


In [33]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))
threshold = od.threshold

New threshold: 0.04146225277142258


In [34]:
X_outlier = [[300,  4,  4,  2,  1,  4,  4,  0,  0,  0, 600,  9]]

In [35]:
od.predict(
    X_outlier
)

{'data': {'instance_score': array([0.06362659]),
  'feature_score': None,
  'is_outlier': array([1])},
 'meta': {'name': 'IForest',
  'detector_type': 'offline',
  'data_type': 'tabular'}}

In [36]:
from alibi_detect.utils.saving import save_detector, load_detector
from os import listdir
from os.path import isfile, join

filepath="ifoutlier"
save_detector(od, filepath) 
onlyfiles = [f for f in listdir(filepath) if isfile(join(filepath, f))]
for filename in onlyfiles:
    print(filename)
    print(get_minio().fput_object(MINIO_MODEL_BUCKET, f"{OUTLIER_MODEL_PATH}/{filename}", join(filepath, filename)))

W1125 19:25:18.982363 140716960044864 saving.py:73] Directory ifoutlier does not exist and is now created.


meta.pickle
('3125d3672c65f418bbc62b18c19442d1-1', None)
IForest.pickle
('74518b501e2c37f3bec392b95163c239-1', None)


## Deploy Seldon Core Model with API

In [55]:
run("kubectl label namespace admin istio-injection=disabled --overwrite", shell=True)

CompletedProcess(args='kubectl label namespace admin istio-injection=disabled --overwrite', returncode=1)

In [75]:
secret = f"""apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
  namespace: seldon-logs
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {MINIO_ACCESS_KEY}
  AWS_SECRET_ACCESS_KEY: {MINIO_SECRET_KEY}
  AWS_ENDPOINT_URL: http://{MINIO_HOST}
  USE_SSL: "false"
"""
with open("secret.yaml","w") as f:
    f.write(secret)
run("cat secret.yaml | kubectl apply -f -", shell=True)

CompletedProcess(args='cat secret.yaml | kubectl apply -f -', returncode=0)

In [78]:
sa = f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
  namespace: seldon-logs
secrets:
  - name: seldon-init-container-secret
"""
with open("sa.yaml","w") as f:
    f.write(sa)
run("kubectl apply -f sa.yaml", shell=True)

CompletedProcess(args='kubectl apply -f sa.yaml', returncode=0)

In [79]:
secret = f"""apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
  namespace: {DEPLOY_NAMESPACE}
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: {MINIO_ACCESS_KEY}
  AWS_SECRET_ACCESS_KEY: {MINIO_SECRET_KEY}
  AWS_ENDPOINT_URL: http://{MINIO_HOST}
  USE_SSL: "false"
"""
with open("secret.yaml","w") as f:
    f.write(secret)
run("cat secret.yaml | kubectl apply -f -", shell=True)

CompletedProcess(args='cat secret.yaml | kubectl apply -f -', returncode=0)

In [81]:
sa = f"""apiVersion: v1
kind: ServiceAccount
metadata:
  name: minio-sa
  namespace: {DEPLOY_NAMESPACE}
secrets:
  - name: seldon-init-container-secret
"""
with open("sa.yaml","w") as f:
    f.write(sa)
run("kubectl apply -f sa.yaml", shell=True)

CompletedProcess(args='kubectl apply -f sa.yaml', returncode=0)

In [82]:
configuration = get_swagger_configuration()
# create an instance of the API class
dep_instance = swagger_client.SeldonDeploymentsApi(swagger_client.ApiClient(configuration))
namespace = 'admin' # str | Namespace provides a logical grouping of resources

Create the Seldon deployment with the Deploy API

In [95]:
from swagger_client.models.seldon_deployment import SeldonDeployment
from swagger_client.models.seldon_deployment_spec import SeldonDeploymentSpec
from swagger_client.models.predictor_spec import PredictorSpec
from swagger_client.models.predictive_unit import PredictiveUnit
from swagger_client.models.logger import Logger
from swagger_client.models.object_meta import ObjectMeta
from swagger_client.models.type_meta import TypeMeta
from swagger_client.models.explainer import Explainer

model_name="income-classifier"
sd = SeldonDeployment(kind="SeldonDeployment",
                      api_version="v1",
                      metadata=ObjectMeta(name=model_name,namespace=namespace),
                      spec=SeldonDeploymentSpec(predictors=[
                          PredictorSpec(graph=PredictiveUnit(implementation="SKLEARN_SERVER",
                                                             model_uri="s3://seldon/sklearn/income/model",
                                                             env_secret_ref_name="seldon-init-container-secret",
                                                             name="classifier",
                                                             logger=Logger(mode="all",url="http://broker-ingress.knative-eventing.svc.cluster.local/seldon-logs/default")),
                                        explainer=Explainer(type="AnchorTabular",
                                                            model_uri="s3://seldon/sklearn/income/explainer",
                                                            env_secret_ref_name="seldon-init-container-secret"),
                                        name="default",
                                        replicas=1)
                      ]))
created = dep_instance.create_seldon_deployment(namespace, sd)

Wait for the Deployment to be ready

In [96]:
state = ""
while not state == "Available":
    res = dep_instance.list_seldon_deployments(namespace)
    for sd in res.items:
        state = sd.status.state
        print(sd.status.state)
    time.sleep(2)
time.sleep(10)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Available


Make a prediction request with the Seldon Deploy API

In [97]:
cookie = authenticate()
payload='{"data": {"ndarray": [[53,4,0,2,8,4,4,0,0,0,60,9]]}}'
cookie_str = f"{KF_SESSION_COOKIE_NAME}={cookie}"
predict_instance = swagger_client.PredictApi(swagger_client.ApiClient(configuration,cookie=cookie_str))
prediction = predict_instance.predict_seldon_deployment(model_name,namespace, prediction={"data": {"ndarray": [[53,4,0,2,8,4,4,0,0,0,60,9]]}})
print(prediction)

{'data': {'names': ['t:0', 't:1'], 'ndarray': [[0.88, 0.12]]}, 'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.5.0-dev'}}}


In [98]:
explain_instance = swagger_client.ExplainerApi(swagger_client.ApiClient(configuration,cookie=cookie_str))
tries = 0
try:
    explanation = explain_instance.explain_seldon_deployment(namespace,model_name,explaindata={"data": {"ndarray": [[53,4,0,2,8,4,4,0,0,0,60,9]]}})
    print(explanation)
except ApiException as e:
    print(e)
    if tries > 5:
        raise e
    print("Retrying")
    tries = tries +1
    time.sleep(5)

{'data': {'anchor': ['Marital Status = Separated', 'Sex = Female'], 'coverage': 0.18063333333333334, 'precision': 0.961218836565097, 'raw': {'all_precision': 0, 'coverage': [0.18063333333333334, 0.18063333333333334], 'examples': [{'covered_false': [[44, 5, 1, 2, 8, 0, 4, 1, 99999, 0, 45, 9], [38, 4, 0, 2, 6, 0, 4, 1, 0, 0, 50, 9], [47, 6, 6, 2, 5, 0, 4, 1, 0, 1977, 50, 9], [64, 6, 6, 2, 5, 0, 4, 1, 0, 0, 9, 9], [45, 4, 3, 2, 2, 1, 4, 1, 0, 2824, 76, 9], [33, 1, 6, 2, 5, 1, 4, 0, 0, 0, 65, 9], [61, 4, 4, 2, 2, 4, 1, 1, 0, 0, 45, 9], [32, 7, 6, 2, 5, 0, 4, 1, 0, 0, 75, 9], [29, 4, 1, 2, 6, 0, 4, 1, 0, 0, 45, 9], [47, 4, 1, 2, 8, 0, 4, 1, 0, 0, 60, 9]], 'covered_true': [[58, 2, 4, 2, 4, 0, 4, 1, 3908, 0, 40, 9], [27, 2, 4, 2, 4, 0, 4, 1, 0, 0, 60, 9], [48, 4, 4, 2, 6, 0, 4, 1, 0, 0, 50, 9], [36, 2, 4, 2, 1, 1, 4, 1, 0, 0, 40, 9], [39, 4, 4, 2, 2, 1, 4, 1, 0, 0, 65, 9], [40, 4, 0, 2, 6, 0, 4, 1, 0, 0, 50, 9], [33, 7, 2, 2, 5, 0, 1, 1, 0, 0, 50, 2], [48, 0, 4, 2, 0, 5, 4, 0, 0, 0, 20, 9], [

## Deploy Outier Detector

In [99]:
configuration = get_swagger_configuration()
outlier = swagger_client.OutlierDetectorApi(swagger_client.ApiClient(configuration))
outlier_params = {
"params": {
    "event_source": "io.seldon.serving.incomeod",
    "event_type": "io.seldon.serving.inference.outlier",
    "http_port": "8080",
    "model_name": "adultod",
    "protocol": "seldon.http",
    "reply_url": "http://hello-display."+namespace,
    "storage_uri": "s3://seldon/sklearn/income/outlier",
    "env_secret_ref": "seldon-init-container-secret"
  }
}
res = outlier.create_outlier_detector_seldon_deployment(model_name, namespace, outlier_detector=outlier_params)

## Deploy KNative Eventing Event Display

In [89]:
event_display=f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-display
  namespace: {DEPLOY_NAMESPACE}
spec:
  replicas: 1
  selector:
    matchLabels: &labels
      app: hello-display
  template:
    metadata:
      labels: *labels
    spec:
      containers:
        - name: event-display
          image: gcr.io/knative-releases/knative.dev/eventing-contrib/cmd/event_display

---

kind: Service
apiVersion: v1
metadata:
  name: hello-display
  namespace: {DEPLOY_NAMESPACE}
spec:
  selector:
    app: hello-display
  ports:
  - protocol: TCP
    port: 80
    targetPort: 8080
"""
with open("event_display.yaml","w") as f:
    f.write(event_display)
run("kubectl apply -f event_display.yaml", shell=True)

CompletedProcess(args='kubectl apply -f event_display.yaml', returncode=0)

In [91]:
run(f"kubectl rollout status -n {DEPLOY_NAMESPACE} deploy/hello-display -n {DEPLOY_NAMESPACE}", shell=True)

CompletedProcess(args='kubectl rollout status -n admin deploy/hello-display -n admin', returncode=0)

## Test Outlier Detection

In [100]:
def predict():
    configuration = get_swagger_configuration()
    cookie = authenticate()
    cookie_str = f"{KF_SESSION_COOKIE_NAME}={cookie}"
    predict_instance = swagger_client.PredictApi(swagger_client.ApiClient(configuration,cookie=cookie_str))
    prediction = predict_instance.predict_seldon_deployment(model_name,namespace, prediction={"data": {"ndarray": [[3000,4,4,2,1,4,4,0,0,0,600,9]]}})
    print(prediction)

In [102]:



def get_outlier_event_display_logs():
    cmd=f"kubectl logs $(kubectl get pod -l app=hello-display -o jsonpath='{{.items[0].metadata.name}}' -n {DEPLOY_NAMESPACE}) -n {DEPLOY_NAMESPACE}"
    ret = Popen(cmd, shell=True,stdout=PIPE)
    res = ret.stdout.read().decode("utf-8").split("\n")
    data= []
    for i in range(0,len(res)):
        if res[i] == 'Data,':
            j = json.loads(json.loads(res[i+1]))
            print(j)
            if "is_outlier"in j["data"].keys():
                data.append(j)
    if len(data) > 0:
        return data[-1]
    else:
        return None
j = None
while j is None:
    predict()
    print("Waiting for outlier logs, sleeping")
    time.sleep(2)
    j = get_outlier_event_display_logs()
    
print(j)
print("Outlier",j["data"]["is_outlier"]==[1])

{'data': {'names': ['t:0', 't:1'], 'ndarray': [[0.92, 0.08]]}, 'meta': {'requestPath': {'classifier': 'seldonio/sklearnserver:1.5.0-dev'}}}
Waiting for outlier logs, sleeping
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': 'IForest', 'detector_type': 'offline', 'data_type': 'tabular'}}
{'data': {'is_outlier': [1]}, 'meta': {'name': '

## Clean Up Resources

In [103]:
outlier.delete_outlier_detector_seldon_deployment(model_name, namespace, _preload_content=False)
dep_instance.delete_seldon_deployment(model_name, namespace, _preload_content=False)

In [107]:
run(f"kubectl delete sa  minio-sa -n {DEPLOY_NAMESPACE}", shell=True)
run(f"kubectl delete sa  minio-sa -n seldon-logs", shell=True)
run(f"kubectl delete secret seldon-init-container-secret -n {DEPLOY_NAMESPACE}", shell=True)
run(f"kubectl delete secret seldon-init-container-secret -n seldon-logs", shell=True)
run(f"kubectl delete deployment hello-display -n {DEPLOY_NAMESPACE}", shell=True)
run(f"kubectl delete svc hello-display -n {DEPLOY_NAMESPACE}", shell=True)
run(f"kubectl delete trigger income-outlier-display -n {DEPLOY_NAMESPACE}", shell=True)

CompletedProcess(args='kubectl delete trigger income-outlier-display -n admin', returncode=1)